In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame
    )

from panda_station import *

In [2]:
"""
builder = DiagramBuilder()
station = PandaStation()


names_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]
station.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)
station.add_panda_with_hand()
plant, scene_graph = station.get_plant_and_scene_graph()

#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))
station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),
                           main_body_name = "base_link", welded = True, name = "A")
station.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),
                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,
                            name = "B"
                           )


station.finalize()

builder.AddSystem(station)
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
station.GetInputPort("panda_position").FixValue(station_context, np.array([np.pi/4, 0.1, 0, -1, 0, 1.5, 0]))
station.GetInputPort("hand_position").FixValue(station_context, [0.08])

meshcat.start_recording()
simulator.AdvanceTo(1.0)

print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSE TABLES")
print(parse_tables(find_resource("directives/three_tables.yaml")))

pose_fluents = [
    ("grasppose", "B", RigidTransform(RotationMatrix(), [0, 0, 0.3])),
    ("atpose", "A", RigidTransform(RotationMatrix(), [0.8, 0.2, 0.2]))
]
update_station(station, station_context, pose_fluents)

simulator_context = simulator.get_context()
plant_context = plant.GetMyContextFromRoot(simulator_context)
station_context = station.GetMyContextFromRoot(simulator_context)

simulator.AdvanceTo(2.0)
meshcat.stop_recording()
meshcat.publish_recording()
"""

'\nbuilder = DiagramBuilder()\nstation = PandaStation()\n\n\nnames_and_links = [(name, "base_link") for name in parse_tables(find_resource("directives/three_tables.yaml"))]\nstation.setup_from_file("directives/three_tables.yaml", names_and_links = names_and_links)\nstation.add_panda_with_hand()\nplant, scene_graph = station.get_plant_and_scene_graph()\n\n#station.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.6, 0, 0.2]))\nstation.add_model_from_file(find_resource("models/manipulands/sdf/foam_brick.sdf"), RigidTransform(RotationMatrix(), [0.8, 0, 0.2]),\n                           main_body_name = "base_link", welded = True, name = "A")\nstation.add_model_from_file(find_resource("models/manipulands/sdf/dumbbell.sdf"), RigidTransform(),\n                           main_body_name = "base_link", P = plant.GetFrameByName("panda_hand"), welded = True,\n                            name = "B"\n                           )\n\n\ns

In [6]:
p = ProblemInfo("/home/agrobenj/drake-tamp/experiments/pick-place-regions/problems/test_problem.yaml")
stations = p.make_all_stations()
print(list(stations.keys()))
builder = DiagramBuilder()
station = stations["main"]
grasp_station = stations["move_free"]
grasp_station_context = grasp_station.CreateDefaultContext()
builder.AddSystem(station)
plant, scene_graph = station.get_plant_and_scene_graph()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()

director = builder.AddSystem(TrajectoryDirector())
builder.Connect(station.GetOutputPort("panda_position_measured"), director.GetInputPort("panda_position"))
builder.Connect(director.GetOutputPort("panda_position_command"), station.GetInputPort("panda_position"))
if (not station.weld_fingers):
    builder.Connect(station.GetOutputPort("hand_state_measured"), director.GetInputPort("hand_state"))
    builder.Connect(director.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
plant_context = plant.GetMyContextFromRoot(simulator_context)

meshcat.start_recording()
simulator.AdvanceTo(0.2)
item = "foam_brick"
grasp_body = station.object_infos[item][0].main_body_info.get_body()
fluent = ("atpose", item, grasp_body.EvalPoseInWorld(plant_context))
print(fluent)
update_station(grasp_station, grasp_station_context, pose_fluents = [fluent], set_others_to_inf = True)

object_info = grasp_station.object_infos[item][0]
body_infos = list(object_info.body_infos.values())
qs = []
costs = []
for body_info in body_infos:
    for shape_info in body_info.get_shape_infos():
        for grasp, cost in find_grasp_q(grasp_station, grasp_station_context, shape_info):
            qs.append(grasp)
            costs.append(cost)
            
q = qs[0]

pregrasp_q, pre_cost = backup_on_hand_z(q, grasp_station, grasp_station_context)
postgrasp_q, post_cost = backup_on_world_z(q, grasp_station, grasp_station_context)
X_HO = q_to_X_HO(q, object_info.main_body_info, grasp_station, grasp_station_context)
print(pregrasp_q, pre_cost)
print(postgrasp_q, post_cost)
print(X_HO)


q1 = np.array([0.0, 0.1, 0, -1.2, 0, 1.6, 0])
panda_traj = find_traj(grasp_station, grasp_station_context, q1, pregrasp_q)
panda_times = np.linspace(0.2, 3.0, len(panda_traj))
panda_traj1 = PiecewisePolynomial.FirstOrderHold(panda_times, panda_traj.T)
panda_traj = np.array([pregrasp_q, q])
panda_times = np.linspace(3.0, 5.0, len(panda_traj))
panda_traj2 = PiecewisePolynomial.FirstOrderHold(panda_times, panda_traj.T)
qh1 = np.array([0])
qh2 = np.array([0.08])
samples = np.array([qh1, qh2])
t = np.array([0.2, 3.0])
hand_traj = PiecewisePolynomial.FirstOrderHold(t, samples.T)
director.add_panda_traj(panda_traj1)
director.add_panda_traj(panda_traj2)
director.add_hand_traj(hand_traj)

simulator.AdvanceTo(5.0)
meshcat.stop_recording()
meshcat.publish_recording()

print(plant.num_positions())
"""
print("PARSING STATION POSES")
print(parse_start_poses(station, station_context))
print("PARSING INITAL CONFIG")
print(parse_config(station, station_context))
print("PARSING TABLES")
print(parse_tables(station.directive))
"""

['main', 'move_free', 'foam_brick', 'meat_can', 'soup_can']
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.
('atpose', 'foam_brick', RigidTransform(
  R=RotationMatrix([
    [0.9999999997624041, 3.159889601463351e-09, -2.1798898692038624e-05],
    [-3.1520043456250145e-09, 0.9999999999999346, 3.6172729175136133e-07],
    [2.1798898693180214e-05, -3.61727222955193e-07, 0.9999999997623387],
  ]),
  p=[0.5000001518097962, -0.19999982772768532, 0.05025528657347098],
))
[-0.25091468  0.28121151 -0.13475748 -2.22379956  0.04498379  2.47304092
 -0.38355543] 1.6987911672117448
[-0.25091292  0.28121572 -0.13475527 -2.2237929   0.04498369  2.47303848
 -0.38355129] 1.6987695535188987
RigidTransform(
  R=RotationMatrix([
    [0.9996638810845998, -0.025925359074599058, 2.4733139498032244e-05],
    [-0.025925358407129024, -0.9996638810449738, -2.6936260300671603e-05],

'\nprint("PARSING STATION POSES")\nprint(parse_start_poses(station, station_context))\nprint("PARSING INITAL CONFIG")\nprint(parse_config(station, station_context))\nprint("PARSING TABLES")\nprint(parse_tables(station.directive))\n'

In [4]:
plant, scene_graph = station.get_plant_and_scene_graph()
plant_context = station.GetSubsystemContext(plant, station_context)
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
panda = station.get_panda()
hand = station.get_hand()
query_output_port = scene_graph.GetOutputPort("query")

def is_colliding(q):
    plant.SetPositions(plant_context, panda, q)
    query_object = query_output_port.Eval(scene_graph_context)
    pairs = query_object.ComputePointPairPenetration()
    print('pairs:')
    for p in pairs:
        print(p.id_A, p.id_B)
    print("panda hand")
    ids = plant.GetBodyIndices(hand)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("panda")
    ids = plant.GetBodyIndices(panda)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    print("brick")
    brick = plant.GetModelInstanceByName("foam_brick")
    ids = plant.GetBodyIndices(brick)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    table = plant.GetModelInstanceByName("table")
    ids = plant.GetBodyIndices(table)
    for body_id in ids:
        body = plant.get_body(body_id)
        for col_id in plant.GetCollisionGeometriesForBody(body):
            print(col_id)
    return query_object.HasCollisions()

def isStateValid(state):
    q = state_to_q(state)
    return not is_colliding(q)

print(is_colliding(q1))

pairs:
<GeometryId value=3> <GeometryId value=156>
<GeometryId value=3> <GeometryId value=157>
<GeometryId value=3> <GeometryId value=158>
<GeometryId value=3> <GeometryId value=159>
<GeometryId value=3> <GeometryId value=167>
<GeometryId value=3> <GeometryId value=169>
<GeometryId value=3> <GeometryId value=171>
<GeometryId value=3> <GeometryId value=173>
<GeometryId value=3> <GeometryId value=178>
<GeometryId value=3> <GeometryId value=180>
<GeometryId value=3> <GeometryId value=182>
<GeometryId value=3> <GeometryId value=184>
panda hand
<GeometryId value=144>
<GeometryId value=147>
<GeometryId value=148>
<GeometryId value=151>
<GeometryId value=152>
panda
<GeometryId value=13>
<GeometryId value=15>
<GeometryId value=17>
<GeometryId value=19>
<GeometryId value=21>
<GeometryId value=23>
<GeometryId value=25>
<GeometryId value=27>
<GeometryId value=29>
<GeometryId value=31>
<GeometryId value=33>
<GeometryId value=35>
<GeometryId value=37>
<GeometryId value=39>
<GeometryId value=43>
<Ge